In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras import Sequential
from keras.layers import Dense
from helper import fn_plot_tf_hist, fn_plot_confusion_matrix
import time

In [ ]:
inpDir = os.path.join( '..', 'input')


modelDir = 'Neural Network'

outDir = os.path.join('output')
subDir = 'fashion_mnist'
altName = 'do_bn_relu_fashion'

# define and set random state 
RANDOM_STATE = 24
np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results
tf.random.set_seed(RANDOM_STATE)

ALPHA = 0.001     # learning rate
EPOCHS = 10
BATCH_SIZE = 64


# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 6),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large',
          'savefig.dpi': 150,
          'image.cmap': 'jet',
          'image.interpolation': 'none',
          'savefig.bbox' : 'tight',
          'lines.linewidth' : 2,
          'legend.numpoints' : 1
         }
CMAP = plt.cm.rainbow
plt.rcParams.update(params);
plt.set_cmap(CMAP);
plt.style.use('seaborn-v0_8-darkgrid') # plt.style.use('ggplot')

TEST_SIZE=0.2

np.set_printoptions(precision=3, suppress=True)

pd.set_option("display.max_columns", 8)
pd.set_option('display.precision', 2)

__all__ = ['np', 'display', 'plt', 'pd', 'sklearn', 'seaborn']

In [ ]:
train_df=pd.read_csv(os.path.join(inpDir, 'fashion-mnist_train.csv'))
test_df=pd.read_csv(os.path.join(inpDir, 'fashion-mnist_test.csv'))

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df.shape, test_df.shape

In [ ]:
class_names = {0: 't-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat',
              5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}

In [ ]:
train_df['label'].unique()

In [ ]:
plot_df = train_df.sample(n = 100)

fig = plt.figure(figsize = (15,12))

i = 0

for _ , row in plot_df.iterrows():
    i += 1
    image = row.values[1:].reshape(28,28)
    ax = fig.add_subplot(10,10,(i), xticks = [], yticks = [])
    ax.imshow(image, cmap = plt.cm.binary,interpolation='nearest')
    ax.text(2,4, str(row.iloc[0]), color = 'b')
    ax.text(2,25, class_names[row.iloc[0]], color = 'r')
plt.tight_layout()
plt.show()

In [ ]:
def fn_plot_label(train_df,test_df):
    plt.figure(figsize = (15,5))
    plt.subplot(1,2,1)
    train_df['label'].value_counts().plot(kind = 'bar', title = 'Train', color = 'DarkBlue', alpha = 0.8)

    plt.subplot(1,2,2)
    test_df['label'].value_counts().plot(kind = 'bar', title = 'Test', color = 'Orange', alpha = 0.8)

In [ ]:
fn_plot_label(train_df,test_df)

In [ ]:
def split_feature_label(row):
    '''
    Args:
        row: array of 785 values
    return: 
        image, label
    '''    
    
    image = tf.reshape(row[1:], [28,28,1])
    
    label = row[0]
    return image, label

In [ ]:
# Creating datasets

tmp_ds = tf.data.Dataset.from_tensor_slices(train_df)

train_ds = tmp_ds.map(split_feature_label)

tmp_ds = tf.data.Dataset.from_tensor_slices(test_df)

test_ds = tmp_ds.map(split_feature_label)

train_ds = train_ds.batch(BATCH_SIZE)

test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
for imgs, labels in train_ds.take(1):
    for i in range(BATCH_SIZE):
        plt.subplot(8, int(BATCH_SIZE // 8), i + 1)
        plt.grid(False)
        plt.imshow(imgs[i].numpy().astype('uint8'), cmap = plt.cm.binary)
        plt.title(class_names[labels[i].numpy()])
        plt.text(2,4, labels[i].numpy(), color = 'b', fontsize = 10)
        plt.axis('off')
plt.tight_layout();

In [ ]:
for imgs, labels in test_ds.take(1):
    for i in range(BATCH_SIZE):
        plt.subplot(8, int(BATCH_SIZE // 8), i + 1)
        plt.grid(False)
        plt.imshow(imgs[i].numpy().astype('uint8'), cmap = plt.cm.binary)
        plt.title(class_names[labels[i].numpy()])
        plt.text(2,4, labels[i].numpy(), color = 'b', fontsize = 10)
        plt.axis('off')
plt.tight_layout();

In [ ]:
normal_layer = tf.keras.layers.Rescaling(1./255.)

img_batch, lbl_batch = next(iter(train_ds))

img = img_batch[0].numpy()

img.max(), img.min()

In [ ]:
norm_ds = train_ds.map(lambda x, y : (normal_layer(x), y))

img_batch, lbl_batch = next(iter(norm_ds))

img = img_batch[0].numpy()

img.max(), img.min()

In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.Sequential()

model.add( tf.keras.layers.Rescaling( 1./255 ))

# Conv 1
model.add(tf.keras.layers.Conv2D(32, (3, 3),activation = 'relu', input_shape = (28,28,1)) )    # 28 x 28 x 32
          
# MaxPool 1
model.add( tf.keras.layers.MaxPool2D((2,2))) # 14 x 14 x 32
          

# Conv 2
model.add( tf.keras.layers.Conv2D(64, (3, 3),
                                activation = 'relu')   )  # 12 x 12 x 64       
          
# MaxPool 2
model.add( tf.keras.layers.MaxPool2D((2,2))) # 6 x 6 x 64
          
# Conv 3
model.add( tf.keras.layers.Conv2D(128, (3, 3),
                                activation = 'relu')    ) # 4 x 4 x 128       
          
# Head
model.add( tf.keras.layers.Flatten()) # 2048
model.add( tf.keras.layers.Dense(128,activation='relu')) # 128
model.add( tf.keras.layers.Dense(10)) # 2048

          

In [ ]:
%load_ext tensorboard

In [ ]:
optimizer = tf.keras.optimizers.Adam()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy( from_logits=True)

model.compile(optimizer=optimizer,loss=loss_fn,metrics=['accuracy'])

In [ ]:
# record in tensorboard

import datetime

log_dir = '..log/fit'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

tb_callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
checkpoint_path = os.path.join(modelDir, subDir, 'all_in.weights.h5')

PATIENCE = 20
LR_FACTOR = 0.1
LR_PATIENCE = 1000
EPOCHS = 1000

checkpt_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor = 'val_loss',
                                                      verbose = 1,
                                                      save_weights_only = True,
                                                      save_best_only = True)

es_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                               patience = PATIENCE,
                                               verbose = 1,
                                               restore_best_weights = True)

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'vall_loss',
                                                   factor = LR_FACTOR,
                                                   patience = LR_PATIENCE,
                                                   verbose = 1)

In [ ]:
history = model.fit(train_ds,
                    epochs=EPOCHS, 
                    validation_data=(test_ds),
                    batch_size= BATCH_SIZE, 
                    verbose=1,
                    callbacks = [checkpt_callback, lr_callback, es_callback],
                    )

In [ ]:
fn_plot_tf_hist(pd.DataFrame(history.history))

In [ ]:
test_loss,test_acc = model.evaluate(test_ds)
test_loss,test_acc

In [ ]:
history.history

In [ ]:
model.summary()

In [ ]:
y_test = tf.concat([y for x,y in test_ds],axis=0).numpy()

y_test.shape

In [ ]:
yhat = model.predict(test_ds)

y_pred = yhat.argmax(axis = 1)


In [ ]:
fn_plot_confusion_matrix(y_test,y_pred,labels=class_names)

In [ ]:
test_df['pred'] = y_pred
plot_df = test_df.sample(n = 50)

In [ ]:
fig = plt.figure(figsize=(15,10))

i = 0

for _ , rows in plot_df.iterrows():
    
    i = i+1
    img = row[1:-1].reshape(28,28)
    color = 'cyan'
    
    if row.iloc[0]!=row.iloc[-1]:
        color = 'red'
    props = dict(boxstyle = 'round', facecolor = color, alpha = 0.5)
        
    ax= fig.add_subplot(5,10,(i),xticks = [],yticks = [])
    ax.imshow(imgs[i].astype('uint8'),cmap = plt.cm.binary)
    ax.set_title(class_names[row.iloc[0]])

plt.axis('off')
plt.tight_layout();